In [ ]:
# Face Cropping

import os
import cv2
lst = ["shivam"]
for i in lst:

  source_folder = f'/content/drive/Shareddrives/CNN_Project/Photos2026/{i}'
  destination_folder = f'/content/drive/Shareddrives/CNN_Project/Photos2026/{i}_Face'

  os.makedirs(destination_folder, exist_ok=True)

  def crop_faces(input_folder, output_folder, face_size=(128, 128)):
      face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

      for img_name in os.listdir(input_folder):
          img_path = os.path.join(input_folder, img_name)

          img = cv2.imread(img_path)
          if img is None:
              continue

          gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

          faces = face_cascade.detectMultiScale(gray, 1.1, 4)

          for (x, y, w, h) in faces:
              face_img = img[y:y + h, x:x + w]
              resized_face = cv2.resize(face_img, face_size)
              output_path = os.path.join(output_folder, img_name)
              cv2.imwrite(output_path, resized_face)
              break

  crop_faces(source_folder, destination_folder)

  print("Face cropping completed. Cropped images saved in the destination folder.")


Face cropping completed. Cropped images saved in the destination folder.


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install facenet-pytorch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from facenet_pytorch import InceptionResnetV1

# Base class to define the basic setup of the model
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()

    def forward(self, x):
        raise NotImplementedError("Forward method not implemented!")


# Define the custom face recognition model class, inheriting from BaseModel
class FaceRecognitionModel(BaseModel):
    def __init__(self, num_classes):
        super(FaceRecognitionModel, self).__init__()

        # Load FaceNet pre-trained model
        self.facenet = InceptionResnetV1(pretrained='vggface2').eval()

        # Freeze FaceNet layers if not training them
        for param in self.facenet.parameters():
            param.requires_grad = False

        # Custom classification layers for fine-tuning
        self.conv1 = nn.Conv2d(512, 256, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 7 * 7, 256)  # Adjust based on input size if different
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        # Pass through FaceNet model to get embeddings
        with torch.no_grad():  # Do not calculate gradients for facenet
            x = self.facenet(x)

        # Pass through additional layers
        x = self.conv1(x.unsqueeze(-1).unsqueeze(-1))  # Adjust shape for Conv2D if needed
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)

        # Flatten for fully connected layers
        x = x.view(x.size(0), -1)  # Flatten

        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        return x


In [23]:
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [28]:
class FaceRecognitionModel(BaseModel):
    def __init__(self, num_classes):
        super(FaceRecognitionModel, self).__init__()
        self.facenet = InceptionResnetV1(pretrained='vggface2').eval()
        for param in self.facenet.parameters():
            param.requires_grad = False

        # Convolutional layers
        self.conv1 = nn.Conv2d(512, 256, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(256, 128, kernel_size=3, padding=1)

        # Placeholder for fc1, updated dynamically
        self.fc1 = None
        self.fc2 = nn.Linear(256, num_classes)  # fc2 remains the same

    def initialize_fc1(self, input_shape):
        # Create a dummy input with the correct shape and device
        dummy_input = torch.randn(input_shape).to(next(self.parameters()).device)

        with torch.no_grad():
            dummy_output = self.facenet(dummy_input)

        # Calculate the flattened size after passing through the convolutional layers
        conv1_output = self.conv1(dummy_output.unsqueeze(-1).unsqueeze(-1))
        conv2_output = self.conv2(conv1_output)
        flat_size = conv2_output.view(conv2_output.size(0), -1).size(1)

        # Initialize fc1 with the calculated input size
        self.fc1 = nn.Linear(flat_size, 256).to(next(self.parameters()).device)




    def forward(self, x):
        if self.fc1 is None:
            raise ValueError("fc1 is not initialized. Call `initialize_fc1` with the input shape.")

        # Move input to the same device as the model parameters
        x = x.to(next(self.parameters()).device)

        with torch.no_grad():
            x = self.facenet(x)

        x = self.conv1(x.unsqueeze(-1).unsqueeze(-1))
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x



    def fine_tune(self, train_loader, val_loader, num_epochs=10, lr=0.001):
        # Define loss and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), lr=lr)

        for epoch in range(num_epochs):
            self.train()
            train_loss = 0.0

            for images, labels in train_loader:
                # Move data to the same device as the model
                device = next(self.parameters()).device
                images, labels = images.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = self(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item() * images.size(0)

            # Validation loop
            self.eval()
            val_loss = 0.0
            correct = 0
            total = 0

            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = self(images)
                    loss = criterion(outputs, labels)

                    val_loss += loss.item() * images.size(0)
                    _, predicted = torch.max(outputs, 1)
                    correct += (predicted == labels).sum().item()
                    total += labels.size(0)

            # Print statistics
            train_loss /= len(train_loader.dataset)
            val_loss /= len(val_loader.dataset)
            val_accuracy = 100 * correct / total

            print(f"Epoch {epoch + 1}/{num_epochs}, "
                  f"Train Loss: {train_loss:.4f}, "
                  f"Validation Loss: {val_loss:.4f}, "
                  f"Validation Accuracy: {val_accuracy:.2f}%")




In [29]:
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class FaceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []
        self.class_to_idx = {}

        for idx, class_name in enumerate(os.listdir(root_dir)):
            self.class_to_idx[class_name] = idx
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                self.images.append(os.path.join(class_dir, img_name))
                self.labels.append(idx)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transforms
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load training and validation data
train_dataset = FaceDataset('/content/drive/Shareddrives/CNN_Project/Attendance_Management/Photos2026/data/train', transform=transform)
val_dataset = FaceDataset('/content/drive/Shareddrives/CNN_Project/Attendance_Management/Photos2026/data/val', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [30]:
num_classes = len(os.listdir('/content/drive/Shareddrives/CNN_Project/Attendance_Management/Photos2026/data/train'))

# Initialize the model
model = FaceRecognitionModel(num_classes=num_classes).to(device)

sample_input_shape = (1, 3, 128, 128)  # Batch size 1, RGB image of size 128x128
model.initialize_fc1(sample_input_shape)

print("Model fc1 layer initialized successfully!")

Model fc1 layer initialized successfully!


In [31]:
# Fine-tune the model
model.fine_tune(train_loader, val_loader, num_epochs=10, lr=0.001)

# Save the fine-tuned model
torch.save(model.state_dict(), 'face_recognition_model.pth')
print("Model fine-tuning complete and saved!")

Epoch 1/10, Train Loss: 2.3836, Validation Loss: 2.4052, Validation Accuracy: 9.09%
Epoch 2/10, Train Loss: 2.2334, Validation Loss: 2.5043, Validation Accuracy: 3.64%
Epoch 3/10, Train Loss: 1.7730, Validation Loss: 3.0141, Validation Accuracy: 5.45%
Epoch 4/10, Train Loss: 1.0620, Validation Loss: 4.4738, Validation Accuracy: 9.09%
Epoch 5/10, Train Loss: 0.5374, Validation Loss: 6.3837, Validation Accuracy: 9.09%
Epoch 6/10, Train Loss: 0.2747, Validation Loss: 8.3308, Validation Accuracy: 9.09%
Epoch 7/10, Train Loss: 0.1498, Validation Loss: 9.7391, Validation Accuracy: 9.09%
Epoch 8/10, Train Loss: 0.1126, Validation Loss: 10.6944, Validation Accuracy: 9.09%
Epoch 9/10, Train Loss: 0.1311, Validation Loss: 10.8173, Validation Accuracy: 9.09%
Epoch 10/10, Train Loss: 0.1015, Validation Loss: 11.0325, Validation Accuracy: 9.09%
Model fine-tuning complete and saved!


In [10]:
import os
import cv2

lst = ["Animesh", "Ankita", "Anmol", "Deependra", "Narayan", "Pankaj", "Sahil", "Sonal", "Suraj", "Vidya"]

for i in lst:
    source_folder = f'/content/drive/Shareddrives/CNN_Project/Attendance_Management/Photos2026/{i}'
    destination_folder = f'/content/drive/Shareddrives/CNN_Project/Photos2026/{i}_Face'
    os.makedirs(destination_folder, exist_ok=True)

    def crop_faces_dnn(input_folder, output_folder, face_size=(128, 128), padding=10):
        # Provide the correct paths to the model and config files
        modelFile = '/content/drive/Shareddrives/CNN_Project/FaceModel/res10_300x300_ssd_iter_140000.caffemodel'
        configFile = '/content/drive/Shareddrives/CNN_Project/FaceModel/deploy.prototxt'
        net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

        for img_name in os.listdir(input_folder):
            img_path = os.path.join(input_folder, img_name)
            img = cv2.imread(img_path)

            if img is None:
                print(f"Could not read image {img_name}")
                continue

            h, w = img.shape[:2]
            blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300), [104, 117, 123], False, False)

            # Detect faces
            net.setInput(blob)
            detections = net.forward()

            for i in range(detections.shape[2]):
                confidence = detections[0, 0, i, 2]
                if confidence > 0.5:  # Adjust threshold for better results
                    box = detections[0, 0, i, 3:7] * [w, h, w, h]
                    (x1, y1, x2, y2) = box.astype("int")

                    # Add padding
                    x1 = max(0, x1 - padding)
                    y1 = max(0, y1 - padding)
                    x2 = min(w, x2 + padding)
                    y2 = min(h, y2 + padding)

                    # Crop face and resize
                    face_img = img[y1:y2, x1:x2]
                    resized_face = cv2.resize(face_img, face_size)

                    # Save cropped face
                    output_path = os.path.join(output_folder, img_name)
                    cv2.imwrite(output_path, resized_face)
                    print(f"Cropped face saved: {output_path}")
                    break  # Process only the first face detected

    crop_faces_dnn(source_folder, destination_folder)
    print(f"Face cropping completed for {i}. Cropped images saved in the destination folder.")


Cropped face saved: /content/drive/Shareddrives/CNN_Project/Photos2026/Animesh_Face/Polish_20240407_134438794.jpg
Cropped face saved: /content/drive/Shareddrives/CNN_Project/Photos2026/Animesh_Face/IMG_20240328_105548.jpg
Cropped face saved: /content/drive/Shareddrives/CNN_Project/Photos2026/Animesh_Face/Polish_20240705_201413287.jpg
Cropped face saved: /content/drive/Shareddrives/CNN_Project/Photos2026/Animesh_Face/1676639882315.png
Cropped face saved: /content/drive/Shareddrives/CNN_Project/Photos2026/Animesh_Face/IMG_20231223_112904.jpg
Cropped face saved: /content/drive/Shareddrives/CNN_Project/Photos2026/Animesh_Face/Polish_20231225_112744894.jpg
Cropped face saved: /content/drive/Shareddrives/CNN_Project/Photos2026/Animesh_Face/IMG_20240321_184308.jpg
Cropped face saved: /content/drive/Shareddrives/CNN_Project/Photos2026/Animesh_Face/IMG_20240303_153544.jpg
Cropped face saved: /content/drive/Shareddrives/CNN_Project/Photos2026/Animesh_Face/IMG_20240525_125222.jpg
Cropped face sav